In [1]:
# device = 'K0303'
# active_power_variable = 'active_power'
path_base_r_string = r'D:\OneDrive - CELSIA S.A E.S.P'

%load_ext autoreload
%autoreload 2
import warnings
# warnings.filterwarnings("ignore")

In [2]:
# this cell enables project path relative imports
import sys
from pathlib import Path
path_base = Path(path_base_r_string)
project_path = path_base / 'Proyectos' / 'Eficiencia_Energetica' / 'Bancolombia' / 'Experimental'
sys.path.append(str(project_path))

In [3]:
# import all your modules here
import json
import pandas as pd
import config as cfg
import library_report as Report

In [4]:
df_devices = Report.get_available_devices("bancolombia")
df_vars = Report.get_available_variables(list(df_devices['id']))

blacklisted = df_vars['variable_label'].isin(cfg.BLACKLISTED_VARIABLE_LABELS)
VAR_IDS_TO_REQUEST = df_vars.loc[~blacklisted, 'variable_id']
VAR_ID_TO_LABEL = dict(zip(df_vars['variable_id'], df_vars['variable_label']))

In [5]:
df_devices

,device_name,id,label
0,BC 793 - Profesionales,62cddcc5b6f878000a3fe353,bc71-2
1,BC 787 - Bocagrande Carrera Tercera,62cddcb346404d045840496e,bc71
2,BC 291 - Las Palmas,629a2452dc4c660b40c0135f,bc-291-las-palmas
3,BC 90 - Megamall,629645af50f5d10259631e85,bc-megamall
4,BC 799 - Floridablanca,62963a5950f5d1025a04f832,bc---floridablanca
5,BC 749 - Santa Monica,627035e3d841544ef198441b,bc-santa-monica
6,BC 750 - Roosevelt,622dfbbff80b95008df697dc,bc58t
7,BC 764 - Jamundí,622b4c9f7ed11a0050150e1d,bc65
8,BC 66 - Palmira,622b4c987ed11a0050150e00,bc64
9,BC 302 - Quebrada Seca,622b4c877ed11a0050150da9,bc61


In [ ]:
CHUNK_SIZE = 10
DATE_INTERVAL_REQUEST = {'start': '2022-06-01', 'end': '2022-09-30'}

df = None
lst_responses = []
for idx in range(0, Report.ceildiv(len(VAR_IDS_TO_REQUEST), CHUNK_SIZE)):
    idx_start = idx * CHUNK_SIZE
    idx_end = (idx + 1) * CHUNK_SIZE
    id_request_subset = VAR_IDS_TO_REQUEST[idx_start:idx_end]

    response = Report.make_request(
        id_request_subset, 
        DATE_INTERVAL_REQUEST, 
    )

    lst_responses.append(response)

df = Report.parse_response(lst_responses, VAR_ID_TO_LABEL)
pd.to_pickle(df, "data_monthly.pkl")

In [ ]:
# df_coords = Report.get_coords_dataframe(list(df_devices['id']))
# pd.to_pickle(df_coords, "data_coordinates.pkl")

In [ ]:
Report.show_response_contents(df)
